In [59]:
# !pip install weaviate-client openai tiktoken langchain sentence-transformers transformers

C:\Users\MANOJKUMAR\anaconda3\lib\threading.py:957: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  del self._target, self._args, self._kwargs
C:\Users\MANOJKUMAR\anaconda3\lib\threading.py:957: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  del self._target, self._args, self._kwargs
C:\Users\MANOJKUMAR\anaconda3\lib\threading.py:957: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  del self._target, self._args, self._kwargs
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1e5c082e380>


In [91]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.vectorstores import Weaviate

Weaviate is an open source vector database. It allows you to store and retrieve data objects based on their semantic properties by indexing them with vectors.

Weaviate can be used stand-alone (aka bring your vectors) or with a variety of modules that can do the vectorization for you and extend the core capabilities.

Weaviate has a GraphQL-API to access your data easily.

In [109]:
import weaviate
client = weaviate.Client(
  url="http://localhost:8080"
)

In [110]:
with open("C:/Users/MANOJKUMAR/Downloads/space_shortened.csv") as csv:
    data = csv.readlines()

In [111]:
data[0].split(',')

['PassengerId',
 'HomePlanet',
 'CryoSleep',
 'Cabin',
 'Destination',
 'Age',
 'VIP',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'Name',
 'Transported\n']

In [108]:
# client.schema.delete_class("Space")

**SCHEMA**

In [112]:
schema= {
    "class":"Space",
    "description":"Holds the information about passengers",
    "properties":[
        {
            "dataType":["text"],
            "description":"Passenger Identification",
            "name":"passengerid"
        },
        {
            "dataType":["text"],
            "description":"Originating Planet",
            "name":"homePlanet"
        },
        {
            "dataType":["text"],
            "description":"Rest of the Info",
            "name":"info"
        }
    ],
    "vectorizer":"text2vec-transformers"
}

In [113]:
client.schema.create_class(schema_class=schema)


**BATCH PROCESS**

In [114]:
with client.batch as batch:
    batch.batch_size=10
    for i, d in enumerate(data):
        properties = {
            "passengerid": d.split(',')[0],
            "homePlanet": d.split(',')[1],
            "info": "".join(d.split(',')[2:]),
        }

        client.batch.add_data_object(properties, "Space")

**QUERIES**

In [116]:
client.data_object.get(limit=1,with_vector=True)['objects']

[{'class': 'Space',
  'creationTimeUnix': 1694002134342,
  'id': '006a98dc-4076-4fd6-855e-239b8a8460f3',
  'lastUpdateTimeUnix': 1694002134342,
  'properties': {'homePlanet': 'Earth',
   'info': 'FalseF/2/STRAPPIST-1e26.0False42.01539.03.00.00.0Billex JacostaffeyTrue\n',
   'passengerid': '0006_01'},
  'vector': [-0.26175418,
   0.21016058,
   -0.045900203,
   0.06447283,
   0.12516785,
   -0.24036497,
   0.2527874,
   0.14465013,
   -0.018116096,
   -0.011559668,
   0.27958098,
   -0.26290333,
   0.0003592735,
   -0.06967009,
   -0.3692205,
   -0.23245429,
   -0.20398703,
   0.050343405,
   -0.10819149,
   0.044729363,
   -0.062115263,
   0.21625984,
   0.103088565,
   0.26383257,
   -0.17983885,
   0.06169569,
   -0.17884286,
   -0.018522592,
   -0.09769848,
   -0.026013903,
   -0.11743281,
   -0.019530155,
   -0.06182133,
   -0.08917204,
   0.16094032,
   0.2510803,
   0.032053705,
   -0.17016579,
   -0.1719693,
   0.096125424,
   -0.119425185,
   -0.288456,
   0.257894,
   0.195901

In [118]:
nearText = {"concepts": ["Earth"]}

result = (
    client.query
    .get("Space", ["passengerid", "homePlanet", "info"])
    .with_near_text(nearText) 
    .with_limit(4)
    .with_additional(["certainty"])
    .do()
)

result

{'data': {'Get': {'Space': [{'_additional': {'certainty': 0.7636065483093262},
     'homePlanet': 'Earth',
     'info': 'FalseF/0/PPSO J318.5-2244.0False0.0483.00.0291.00.0Sandie HinetthewsTrue\n',
     'passengerid': '0005_01'},
    {'_additional': {'certainty': 0.7409041225910187},
     'homePlanet': 'Earth',
     'info': 'TrueG/0/STRAPPIST-1e28.0False0.00.00.00.0Candra JacostaffeyTrue\n',
     'passengerid': '0006_02'},
    {'_additional': {'certainty': 0.7365733981132507},
     'homePlanet': 'Earth',
     'info': 'FalseTRAPPIST-1e31.0False32.00.0876.00.00.0Justie PoolesFalse',
     'passengerid': '0012_01'},
    {'_additional': {'certainty': 0.7255687713623047},
     'homePlanet': 'Earth',
     'info': 'FalseF/2/PTRAPPIST-1e28.0False8.0974.012.02.07.0Elle BertsontryTrue\n',
     'passengerid': '0011_01'}]}}}

In [100]:
client.data_object.get(limit=1,class_name='Space',with_vector=False)['objects']

[{'class': 'Space',
  'creationTimeUnix': 1693995683392,
  'id': '1f29951e-7375-48ef-a7a4-1a9f073c4419',
  'lastUpdateTimeUnix': 1693995683392,
  'properties': {'homePlanet': 'Europa',
   'info': 'FalseB/1/P55 Cancri e45.0False39.07295.0589.0110.0124.0Wezena FlaticTrue\n',
   'passengerid': '0008_03'},
  'vectorWeights': None}]

**Get with near Vector**

In [119]:
import json
result = (
    client.query
    .get("Space", ["info", "homePlanet","passengerid"])
    .with_near_vector({
         'vector': [-0.24135287,
                           0.071159266,
                           0.018247215,
                           -0.02124098,
                           0.07014508,
                           -0.19844617,
                           0.23468176,
                           0.12234638,
                           0.020412004,
                           -0.013041719,
                           0.36520362,
                           -0.48155668,
                           -0.08002021,
                           -0.20024481,
                           -0.27238402,
                           -0.30132198,
                           -0.37794653,
                           -0.07357164,
                           -0.04312449,
                           0.10038683,
                           0.040369637,
                           0.36672696,
                           -0.08689905,
                           0.13522844,
                           0.013063283,
                           0.15066057,
                           -0.1325,
                           -0.12711121,
                           0.05007553,
                           0.010442145,
                           0.040475745,
                           0.07216921,
                           -0.11920103,
                           -0.14781675,
                           0.42805812,
                           0.23691346,
                           -0.015157905,
                           -0.10408438,
                           -0.07028142,
                           0.1705254,
                           -0.051122442,
                           -0.37868106,
                           0.21395057,
                           -0.10124817,
                           0.015756192,
                           0.090080716,
                           -0.14558415,
                           0.2264228,
                           0.16264212,
                           -0.06614403,
                           -0.0059978263,
                           0.14628002,
                           -0.24892856,
                           0.01601454,
                           0.0709796,
                           -0.01929039,
                           0.025903901,
                           -0.089375556,
                           0.039183903,
                           -0.10955643,
                           0.17545958,
                           -0.090541445,
                           -0.05989067,
                           -0.021027304,
                           0.18416433,
                           0.15046373,
                           -0.1051378,
                           -0.4532144,
                           0.10952419,
                           0.12116425,
                           -0.08627176,
                           -0.098034255,
                           -0.2730632,
                           0.010895117,
                           -0.0062899105,
                           0.016875057,
                           0.04555743,
                           0.2908366,
                           0.2934363,
                           -0.30786642,
                           -0.089308076,
                           -0.4084073,
                           -0.13373211,
                           -0.03422338,
                           0.44262716,
                           -0.012702168,
                           0.015484751,
                           0.037691385,
                           0.01445953,
                           -0.16983637,
                           0.25711077,
                           -0.07293164,
                           0.21610275,
                           0.3093849,
                           -0.13467817,
                           0.028570473,
                           -0.06776007,
                           -0.15630208,
                           0.12893368,
                           0.298574,
                           0.108449936,
                           -0.06603149,
                           -0.41090956,
                           0.11241115,
                           -0.32319003,
                           0.024190407,
                           -0.024886424,
                           0.13651071,
                           0.18559362,
                           0.039449066,
                           0.15927999,
                           -0.27724797,
                           0.00032201927,
                           -0.023649035,
                           -0.23709774,
                           -0.15806776,
                           0.16583148,
                           0.03488772,
                           -0.08491759,
                           -0.30065367,
                           0.04175867,
                           -0.019167889,
                           0.029451057,
                           0.34854695,
                           -0.21105485,
                           -0.40220484,
                           0.12266246,
                           -5.8289337e-31,
                           0.109335795,
                           -0.1500971,
                           0.19350491,
                           -0.37911457,
                           -0.29942775,
                           0.07039456,
                           -0.20309198,
                           0.18266678,
                           0.062116094,
                           -0.071602605,
                           -0.08662992,
                           -0.17241122,
                           -0.16488747,
                           -0.14142512,
                           -0.24156512,
                           -0.09719115,
                           0.3183559,
                           0.15974925,
                           -0.1350694,
                           -0.20006573,
                           0.12864825,
                           0.3331212,
                           0.055976402,
                           -0.05872313,
                           0.25641444,
                           0.28266948,
                           -0.22759268,
                           -0.2704768,
                           -0.069538146,
                           0.1743984,
                           -0.02810375,
                           -0.115560345,
                           0.110476635,
                           -0.1699677,
                           0.10211487,
                           0.123723984,
                           -0.16997646,
                           -0.053618215,
                           -0.26116398,
                           0.08920107,
                           -0.035639673,
                           -0.026115214,
                           0.013136133,
                           -0.06332945,
                           0.30020314,
                           -0.14676416,
                           0.15511358,
                           0.03344974,
                           0.31230035,
                           0.20732619,
                           -0.08742102,
                           0.123450086,
                           -0.2641089,
                           0.018299328,
                           0.21867254,
                           0.036708158,
                           0.17386615,
                           0.07334849,
                           -0.004610194,
                           0.15852802,
                           0.14070512,
                           0.09754359,
                           0.2901503,
                           -0.12722157,
                           -0.21174787,
                           0.07898269,
                           -0.16750841,
                           -0.1259508,
                           -0.07666553,
                           0.12467014,
                           -0.08293117,
                           -0.16872154,
                           0.23826869,
                           0.009871089,
                           0.09163537,
                           -0.2916637,
                           -0.056172423,
                           -0.022522438,
                           -0.1989123,
                           0.053624842,
                           -0.16578758,
                           0.20433728,
                           0.049595784,
                           -0.38308924,
                           -0.19095735,
                           -0.101290174,
                           -0.18239811,
                           0.17699003,
                           -0.061209545,
                           -0.21845818,
                           0.19307633,
                           -0.1646905,
                           -0.1656762,
                           -0.18489304,
                           -0.309606,
                           -1.9236235e-32,
                           -0.23115556,
                           0.11594839,
                           0.0024617987,
                           0.14885788,
                           -0.13845833,
                           -0.09925654,
                           -0.14853068,
                           0.35508534,
                           0.080400616,
                           0.22653289,
                           0.43049645,
                           0.21513827,
                           -0.10947972,
                           -0.45201558,
                           0.07141371,
                           0.1757327,
                           0.47404042,
                           0.07551785,
                           -0.32036927,
                           0.16999811,
                           0.07093092,
                           0.12827276,
                           0.12878333,
                           0.030956684,
                           -0.044234894,
                           0.059919648,
                           0.19488175,
                           -0.047666155,
                           -0.09767532,
                           0.02025778,
                           0.12711012,
                           0.2802026,
                           -0.13353506,
                           0.39036664,
                           0.09498434,
                           -0.042404257,
                           0.08570408,
                           0.04420292,
                           -0.16146712,
                           -0.18828064,
                           0.09912793,
                           0.2049243,
                           0.14375485,
                           0.11017548,
                           -0.0059872167,
                           -0.22257547,
                           0.29975426,
                           0.19255507,
                           0.047448184,
                           0.06961446,
                           0.1933062,
                           -0.28940287,
                           0.22337092,
                           0.21297313,
                           0.12536742,
                           0.27780747,
                           -0.3032079,
                           0.2083375,
                           0.114098154,
                           -0.014137136,
                           0.27130458,
                           -0.25874436,
                           0.040477276,
                           -0.00571484,
                           0.19845286,
                           -0.17913786,
                           -0.2590626,
                           0.49602634,
                           0.08541447,
                           0.22401628,
                           0.08960414,
                           0.004836344,
                           -0.11343545,
                           -0.38334656,
                           -0.044908434,
                           0.09385181,
                           0.063606404,
                           0.016952239,
                           -0.10472304,
                           0.0046196864,
                           0.1979136,
                           0.1009729,
                           -0.08129436,
                           -0.014334615,
                           0.10541298,
                           -0.11264907,
                           -0.11174291,
                           -0.16238743,
                           -0.1869196,
                           0.18636554,
                           -0.19898309,
                           -0.026248131,
                           -0.11660967,
                           0.24385199,
                           0.03175532,
                           -6.410679e-33,
                           0.1556319,
                           -0.035251357,
                           -0.11586547,
                           -0.04095237,
                           0.23511438,
                           -0.27863595,
                           -0.17823303,
                           -0.20170176,
                           -0.119356945,
                           -0.1675235,
                           0.1632956,
                           -0.005704092,
                           -0.06740534,
                           -0.2805755,
                           0.061329965,
                           -0.12920739,
                           -0.36825913,
                           0.08701687,
                           -0.045191023,
                           0.051818553,
                           0.05842585,
                           -0.041035555,
                           0.04771862,
                           -0.11609368,
                           -0.058741227,
                           0.13308781,
                           -0.0027580247,
                           0.35559136,
                           0.10542971,
                           -0.36447194,
                           -0.06834668,
                           -0.1170147,
                           -0.087552235,
                           0.089157075,
                           -0.21760803,
                           0.0036876292,
                           -0.06598967,
                           0.27854747,
                           -0.056069415,
                           -0.15952305,
                           -0.120838515,
                           0.06669415,
                           -0.0006272009,
                           0.18222126,
                           0.1692978,
                           0.08682538,
                           -0.13124926,
                           -0.04466002,
                           -0.09784846,
                           -0.20177498,
                           -0.14747028,
                           0.19947578,
                           -0.040854134,
                           -0.15889348,
                           0.020465763,
                           -0.08674786,
                           -0.05651897,
                           0.04817598,
                           -0.30572227,
                           0.13233402,
                           -0.10491209,
                           -0.01747879,
                           -0.18872993,
                           0.1967703],

        "certainty": 0.7
    })
    .with_limit(2)
    .do()
)

print(json.dumps(result, indent=4))

{
    "data": {
        "Get": {
            "Space": [
                {
                    "homePlanet": "Earth",
                    "info": "FalseF/2/PTRAPPIST-1e28.0False8.0974.012.02.07.0Elle BertsontryTrue\n",
                    "passengerid": "0011_01"
                },
                {
                    "homePlanet": "Earth",
                    "info": "FalseF/0/PPSO J318.5-2244.0False0.0483.00.0291.00.0Sandie HinetthewsTrue\n",
                    "passengerid": "0005_01"
                }
            ]
        }
    }
}


**Filtering Data**

In [121]:
where_filter = {
    "path": ["homePlanet"],
    "operator": "Equal",
    "valueText": "Earth",
}

In [122]:
nearText = {"concepts": ["Trappist"]}

result_earth = (
    client.query
    .get("Space", ["passengerid", "homePlanet"])
    .with_near_text(nearText) #this method conv text to vector
    .with_where(where_filter)
    .do()
)

print(json.dumps(result_earth, indent=4))


{
    "data": {
        "Get": {
            "Space": [
                {
                    "homePlanet": "Earth",
                    "passengerid": "0002_01"
                },
                {
                    "homePlanet": "Earth",
                    "passengerid": "0006_01"
                },
                {
                    "homePlanet": "Earth",
                    "passengerid": "0010_01"
                },
                {
                    "homePlanet": "Earth",
                    "passengerid": "0007_01"
                },
                {
                    "homePlanet": "Earth",
                    "passengerid": "0004_01"
                },
                {
                    "homePlanet": "Earth",
                    "passengerid": "0006_02"
                },
                {
                    "homePlanet": "Earth",
                    "passengerid": "0011_01"
                },
                {
                    "homePlanet": "Earth",
        

**Aggregation**

In [123]:
result_agg = (
    client.query
    .aggregate("Space")
    .with_fields("meta {count}")
    .do()
)

print(json.dumps(result_agg, indent=4))

{
    "data": {
        "Aggregate": {
            "Space": [
                {
                    "meta": {
                        "count": 17
                    }
                }
            ]
        }
    }
}


In [105]:
result_agg = (
    client.query
    .aggregate("Space")
    .with_fields("meta {count}")
    .with_where(where_filter)
    .do()
)

print(json.dumps(result_agg, indent=4))

{
    "data": {
        "Aggregate": {
            "Space": [
                {
                    "meta": {
                        "count": 8
                    }
                }
            ]
        }
    }
}


In [106]:
vectorstore = Weaviate(client, 
                       "Space", 
                       "homePlanet")

In [124]:
vectorstore.similarity_search("humans")


[Document(page_content='Earth', metadata={}),
 Document(page_content='Earth', metadata={}),
 Document(page_content='Earth', metadata={}),
 Document(page_content='Earth', metadata={})]